In [8]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_regression

In [18]:
# all_data = pd.read_excel('all_data.xlsx').drop('Unnamed: 0', axis=1)

In [15]:
# g_curve = pd.DataFrame()
# for year in range(2020, 2024):
#     g_curve_tmp = pd.read_excel('/Users/veronikayashina/Desktop/мага/РМ/проект 2/portfolio-risk/data/кривая бескупонной доходности.xlsx', sheet_name = str(year))
#     g_curve_tmp.columns = ['date', 'datetime' , 'year_1', 'year_3','year_5', 'year_10', 'year_15', 'year_20']
#     g_curve_tmp['date'] = g_curve_tmp['date'].astype('str')
#     g_curve = pd.concat([g_curve, g_curve_tmp], ignore_index = True)
# all_data = all_data.merge(g_curve, on = ['date'], how = 'inner')

In [19]:
# all_data.drop('datetime', axis=1).to_csv('../data/all_data.csv')

In [20]:
# all_data = pd.read_csv('../data/all_data.csv')

In [22]:
# all_data.columns

In [26]:
all_data.head()

,date,su26230_days_before_coupon,su26224_days_before_coupon,su26222_days_before_coupon,su26221_days_before_coupon,su26218_days_before_coupon,ecb_rate,aluminum,brent,cbr_key_rate,eur_rub,moex_index,nickel,rtsi,usd_rub,su26218,su26221,su26222,su26224,su26230,gazp,gmkn,lkoh,magn,mgnt,moex,rosn,rual,sber,vtbr,year_1,year_3,year_5,year_10,year_15,year_20,pca_cbd
0,2020-01-10,89,145,103,89,82,0.25,1806.0,65.02,6.25,68.0555,3123.66,14116.0,1614.69,61.2340,119.766,113.450,105.569,106.199,115.049,251.9,198.78,6490.0,43.345,3319.5,112.99,473.50,31.70,258.19,0.04760,5.48,5.69,5.90,6.26,6.48,6.61,-5.601225
1,2020-01-11,89,145,103,89,82,0.25,1806.0,65.02,6.25,68.0555,3123.66,14116.0,1614.69,61.2340,119.766,113.450,105.569,106.199,115.049,251.9,198.78,6490.0,43.345,3319.5,112.99,473.50,31.70,258.19,0.04760,5.48,5.69,5.90,6.26,6.48,6.61,-5.601225
2,2020-01-12,89,145,103,89,82,0.25,1806.0,65.02,6.25,68.0555,3123.66,14116.0,1614.69,61.2340,119.766,113.450,105.569,106.199,115.049,251.9,198.78,6490.0,43.345,3319.5,112.99,473.50,31.70,258.19,0.04760,5.48,5.69,5.90,6.26,6.48,6.61,-5.601225
3,2020-01-13,86,142,100,86,79,0.25,1798.0,64.23,6.25,68.0450,3151.69,14007.5,1619.73,61.2632,119.999,113.791,105.696,106.203,115.700,251.5,203.62,6530.0,44.610,3345.0,114.10,481.15,32.20,262.40,0.04806,5.45,5.64,5.84,6.18,6.39,6.52,-5.760549
4,2020-01-14,85,141,99,85,78,0.25,1809.0,64.42,6.25,67.8162,3129.77,13781.0,1604.96,60.9474,119.800,113.597,105.520,106.293,115.550,250.0,200.00,6520.5,44.280,3339.0,114.02,475.50,32.83,259.05,0.04726,5.46,5.68,5.89,6.26,6.47,6.60,-5.626252


In [14]:
risk_factors_dtc ={
'sber': ['moex_index', 'rtsi', 'pca_cbd', 'ecb_rate'], #сбер
'vtbr': ['moex_index', 'pca_cbd'], #втб
'rual': ['moex_index', 'pca_cbd'], #русал
'rosn': ['moex_index', 'pca_cbd', 'brent', 'eur_rub', 'usd_rub'], #роснефть
'moex': ['moex_index', 'pca_cbd'], #мосбиржа
'mgnt': ['moex_index', 'pca_cbd', 'eur_rub', 'usd_rub'], #магнит
'magn': ['moex_index', 'pca_cbd', 'eur_rub', 'usd_rub'], #ММК
'lkoh': ['moex_index', 'pca_cbd', 'eur_rub', 'usd_rub'], #лукойл
'gmkn': ['moex_index', 'pca_cbd', 'eur_rub', 'usd_rub'], #гмк норникель
'gazp': ['moex_index', 'pca_cbd', 'eur_rub', 'usd_rub', 'brent'] #газпром

}

In [63]:
def calc_mutual_info(y, X):
    """
    Selects top N features by Mutual Information with target
    """
    # determine the mutual information
    mutual_info = mutual_info_regression(X.fillna(0), y)
    mutual_info = pd.Series(mutual_info)
    mutual_info.index = X.columns
    return list(mutual_info.sort_values(ascending=False).index)

In [25]:
def select_risk_factors(factor, risk_factors_list, top_factors=4):
    """
    Функция принимает на вход колонку-название актива и список его возможных риск-факторов из словаря 
    Отсеиваются факторы с корреляцией между фактором и таргетом < 0.05
    Смотрится взаимная корреляция факторов, если она превышает 0.9, то оставляется один фактор с наибольшей MI с таргетом
    """
    target = all_data[factor].values
    risk_factors = all_data[risk_factors_list]
    drop_list_idx = []
    corr_list = []
    #считаем обычную корреляцию с таргетом
    for i in range(len(risk_factors.columns)):
        corr_with_target = np.corrcoef(x = risk_factors[risk_factors_list[i]].values, y = target)[0][1]
        corr_list.append(corr_with_target)
        if corr_with_target < 0.05: #дропаем, если меньше 0.05
            drop_list_idx.append(i)
    
    #ранжируем фичи по MI с таргетом
    top_mutual_information_list = calc_mutual_info(target, risk_factors)

    #смотрим взаимную корреляцию фичей
    mutual_correlation = np.corrcoef(x = risk_factors, rowvar=False)
    a = np.where(mutual_correlation >= 0.9)[0]
    b = np.where(mutual_correlation >= 0.9)[1]
    correlated_features = []
    for i in range(len(a)):
        if a[i]!= b[i]:
            correlated_features.append(tuple(sorted([a[i], b[i]])))
    correlated_features = set(correlated_features) #тут сет пар индексов скоррелированных фичей

    for pair in correlated_features: #смотрим, какая из фичей из пары на каком месте в ранжированном списке MI
        mi_1 =np.where(np.array(top_mutual_information_list) == risk_factors.columns[pair[0]]) 
        mi_2 =np.where(np.array(top_mutual_information_list) == risk_factors.columns[pair[1]])

        if mi_1 > mi_2: #если первый признак из пары менее связан с таргетом (дальше от начала списка MI)
            drop_list_idx.append(pair[0]) #дропаем первый признак
        else:
            drop_list_idx.append(pair[1])
    
    drop_list_names = list(risk_factors[risk_factors.columns[drop_list_idx]].columns)
    print(drop_list_names)
    return all_data[[factor] + risk_factors_list].drop(drop_list_names, axis=1)
        

In [79]:
select_risk_factors('rosn', risk_factors_dtc['rosn'])

['usd_rub']


,rosn,brent,eur_rub
0,473.50,65.02,68.0555
1,481.15,64.23,68.0450
2,475.50,64.42,67.8162
3,474.50,64.00,68.4213
4,480.95,64.62,68.3747
...,...,...,...
884,577.95,79.39,99.0480
885,583.20,79.07,100.5113
886,591.45,79.54,101.1178
887,593.95,77.15,101.3451
